## Druga część projektu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### wczytanie wszystkich zbiorow danych

### podział na zbiór testowy i treningowy

### prosty model regresji nr 1

### prosty model regresji nr 2 